# Allenamento in Google Colab di LwCoronNet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
PATH_FOLDER = '/content/drive/MyDrive/Colab Notebooks/Pneumonia Detection/'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets
import os
import matplotlib.pyplot as plt

# Parametri dell'allenamento
BATCH_SIZE = 32
LEARNING_RATE = 0.0005
EPOCHS = 100
OPTIMIZER = 'Adam'
LOSS_FUNCTION = 'Categorical Cross-Entropy'
H, W = 224, 244

# Transformation for Data Augmentation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.9, 1.1)),
    transforms.Resize((H, W)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: add_gaussian_noise(x) if torch.rand(1).item() > 0.5 else x),
])

# Dataset e DataLoader
path_to_train = os.path.join(PATH_FOLDER, "Data", "train")
path_to_test = os.path.join(PATH_FOLDER, "Data", "test")
train_dataset = datasets.ImageFolder(path_to_train, transform=transform)
test_dataset = datasets.ImageFolder(path_to_test, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


class LwCoronNet(nn.Module):
    def __init__(self, num_classes=3):
        super(LwCoronNet, self).__init__()

        # Primo blocco convoluzionale
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5, stride=2)
        self.norm1 = nn.BatchNorm2d(num_features=64)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(3, stride=3)

        # Secondo blocco convoluzionale
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.norm2 = nn.BatchNorm2d(num_features=128)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(3, stride=3)

        # Terzo blocco convoluzionale
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3)
        self.norm3 = nn.BatchNorm2d(num_features=256)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(3, stride=3)

        # Layer completamente connessi
        self.flatten = nn.Flatten()

        self.norm4 = nn.BatchNorm1d(num_features=2304)
        self.drop4 = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(in_features=2304, out_features=128)  # Inizializzazione per immagini 224x224

        self.relu5 = nn.ReLU()

        # Blocchi aggiuntivi di normalizzazione e dropout
        self.norm5 = nn.BatchNorm1d(num_features=128)
        self.drop5 = nn.Dropout(p=0.5)

        self.fc2 = nn.Linear(in_features=128, out_features=num_classes)

    def forward(self, x):
        # Passaggio attraverso i blocchi convoluzionali
        x = self.conv1(x)
        x = self.norm1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.norm2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = self.conv3(x)
        x = self.norm3(x)
        x = self.relu3(x)
        x = self.pool3(x)

        # Layer completamente connessi
        x = self.flatten(x)
        x = self.norm4(x)
        x = self.drop4(x)
        x = self.fc1(x)

        x = self.relu5(x)

        # Passaggio attraverso i layer aggiuntivi
        x = self.norm5(x)
        x = self.drop5(x)

        x = self.fc2(x)
        return x


# Funzione per aggiungere rumore gaussiano
def add_gaussian_noise(img, mean=0, std=0.25):
    noise = torch.randn_like(img) * std + mean
    noisy_img = img + noise
    noisy_img = torch.clamp(noisy_img, 0., 1.)  # Assicuriamoci che i valori siano tra 0 e 1
    return noisy_img


# Funzione di allenamento
def train_net(model, train_loader, optimizer, criterion, epochs, save_dir='./training_results'):
    # Creazione della cartella per salvare i modelli e grafici
    os.makedirs(save_dir, exist_ok=True)

    history = {'loss': [], 'accuracy': []}
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct, total = 0, 0

        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total
        history['loss'].append(epoch_loss)
        history['accuracy'].append(epoch_acc)

        # Stampa e salvataggio ogni epoca
        message = f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}"
        print(message)

        # Salvataggio del messaggio nel file di log
        log_file_path = os.path.join(save_dir, "training_log.txt")
        save_log_to_file(log_file_path, message)

        # Salvataggio del modello ad ogni epoca
        torch.save(model.state_dict(), os.path.join(save_dir, f"model_epoch_{epoch + 1}.pth"))

        # Salvataggio dei grafici alla fine di ogni epoca
        plot_training_curves(history, save_dir, epoch + 1)

    return history


def save_log_to_file(file_path, message):
    """
    Salva un messaggio in un file di testo. Crea il file se non esiste.

    Args:
        file_path (str): Il percorso del file in cui salvare il messaggio.
        message (str): Il messaggio da salvare.
    """
    # Verifica se il file esiste, altrimenti crealo
    if not os.path.exists(file_path):
        with open(file_path, 'w') as file:  # 'w' crea il file vuoto
            pass  # File creato vuoto, pronto per l'aggiunta di messaggi

    # Aggiunge il messaggio al file
    with open(file_path, 'a') as file:
        file.write(message + '\n')


# Funzione per i grafici
def plot_training_curves(history, save_dir, epoch):
    epochs = range(1, len(history['loss']) + 1)  # Epoche da 1 a N

    # Creazione dei grafici
    plt.figure(figsize=(12, 6))

    # Diagramma della perdita
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history['loss'], label='Training Loss', color='red', marker='o')
    plt.title('Loss Diagram')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.legend()

    # Diagramma dell'accuratezza
    plt.subplot(1, 2, 2)
    plt.plot(epochs, history['accuracy'], label='Training Accuracy', color='blue', marker='o')
    plt.title('Accuracy Diagram')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.grid(True)
    plt.legend()

    plt.tight_layout()

    # Salvataggio dei grafici
    plt.savefig(os.path.join(save_dir, f"training_curves_epoch_{epoch}.png"))
    plt.close()

In [ ]:
# Definizione dell'ottimizzatore e della loss function
model = LwCoronNet()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

# Allenamento della rete
train_net(model, train_loader, optimizer, criterion, epochs=EPOCHS, save_dir=os.path.join(PATH_FOLDER, 'training_results'))

Epoch 1/100, Loss: 0.7160, Accuracy: 0.7234
Epoch 2/100, Loss: 0.4571, Accuracy: 0.8542
Epoch 3/100, Loss: 0.3550, Accuracy: 0.8871
Epoch 4/100, Loss: 0.3108, Accuracy: 0.8946
Epoch 5/100, Loss: 0.2894, Accuracy: 0.9014
Epoch 6/100, Loss: 0.2694, Accuracy: 0.9114
Epoch 7/100, Loss: 0.2468, Accuracy: 0.9121
Epoch 8/100, Loss: 0.2274, Accuracy: 0.9182
Epoch 9/100, Loss: 0.2175, Accuracy: 0.9230
Epoch 10/100, Loss: 0.2114, Accuracy: 0.9285
Epoch 11/100, Loss: 0.2006, Accuracy: 0.9304
Epoch 12/100, Loss: 0.1952, Accuracy: 0.9318
Epoch 13/100, Loss: 0.2023, Accuracy: 0.9347
Epoch 14/100, Loss: 0.1862, Accuracy: 0.9360
Epoch 15/100, Loss: 0.1866, Accuracy: 0.9360
Epoch 16/100, Loss: 0.1817, Accuracy: 0.9357
Epoch 17/100, Loss: 0.1774, Accuracy: 0.9392


# Applicazione tecniche di OverSampling o DownSampling, per dati sbilanciati

In [ ]:
from torch.utils.data import WeightedRandomSampler, Subset
import numpy as np

# Funzione per il downsampling
def apply_downsampling(dataset, class_counts):
    """
    Riduce il dataset downsampling le classi maggioritarie per bilanciare le proporzioni.
    Args:
        dataset (Dataset): Dataset completo.
        class_counts (dict): Dizionario con la distribuzione delle classi nel formato {classe: conteggio}.
    Returns:
        Subset: Dataset ridotto.
    """
    min_count = min(class_counts.values())  # Dimensione della classe minoritaria
    indices = []
    for class_idx, count in class_counts.items():
        class_indices = [i for i, (_, label) in enumerate(dataset) if label == class_idx]
        downsampled_indices = np.random.choice(class_indices, min_count, replace=False)
        indices.extend(downsampled_indices)
    return Subset(dataset, indices)

# Funzione per l'oversampling
def apply_oversampling(dataset, class_counts):
    """
    Applica l'oversampling per le classi minoritarie utilizzando WeightedRandomSampler.
    Args:
        dataset (Dataset): Dataset completo.
        class_counts (dict): Dizionario con la distribuzione delle classi nel formato {classe: conteggio}.
    Returns:
        WeightedRandomSampler: Sampler pesato per DataLoader.
    """
    class_weights = [1.0 / class_counts[label] for _, label in dataset]
    sampler = WeightedRandomSampler(weights=class_weights, num_samples=len(dataset), replacement=True)
    return sampler

# Calcolo della distribuzione delle classi
def compute_class_distribution(dataset):
    """
    Calcola la distribuzione delle classi in un dataset.
    Args:
        dataset (Dataset): Dataset completo.
    Returns:
        dict: Distribuzione delle classi {classe: conteggio}.
    """
    class_counts = {}
    for _, label in dataset:
        class_counts[label] = class_counts.get(label, 0) + 1
    return class_counts

# Applicazione delle strategie
class_counts = compute_class_distribution(train_dataset)
print("Distribuzione originale delle classi:", class_counts)

# Scegli una strategia
strategy = "downsampling"  # Cambia a "downsampling" per valutare il downsampling

if strategy == "downsampling":
    print("Applicazione del downsampling...")
    train_dataset = apply_downsampling(train_dataset, class_counts)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
elif strategy == "oversampling":
    print("Applicazione dell'oversampling...")
    sampler = apply_oversampling(train_dataset, class_counts)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler)

# Rete e allenamento
model = LwCoronNet()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

# Allenamento con la strategia scelta
train_net(model, train_loader, optimizer, criterion, epochs=EPOCHS, save_dir=os.path.join(PATH_FOLDER, f'training_results_{strategy}'))


Distribuzione originale delle classi: {0: 460, 1: 1266, 2: 3418}
Applicazione del downsampling...
Epoch 1/100, Loss: 0.7785, Accuracy: 0.6377
Epoch 2/100, Loss: 0.6238, Accuracy: 0.7616
Epoch 3/100, Loss: 0.5129, Accuracy: 0.8065
Epoch 4/100, Loss: 0.4986, Accuracy: 0.8036
Epoch 5/100, Loss: 0.4454, Accuracy: 0.8304
Epoch 6/100, Loss: 0.4479, Accuracy: 0.8275
Epoch 7/100, Loss: 0.4109, Accuracy: 0.8551
Epoch 8/100, Loss: 0.3742, Accuracy: 0.8565
Epoch 9/100, Loss: 0.3358, Accuracy: 0.8826
Epoch 10/100, Loss: 0.3013, Accuracy: 0.8841
Epoch 11/100, Loss: 0.3101, Accuracy: 0.8935
Epoch 12/100, Loss: 0.3370, Accuracy: 0.8761
Epoch 13/100, Loss: 0.3006, Accuracy: 0.8964
Epoch 14/100, Loss: 0.3133, Accuracy: 0.8913
Epoch 15/100, Loss: 0.2931, Accuracy: 0.8964
Epoch 16/100, Loss: 0.2725, Accuracy: 0.9022
Epoch 17/100, Loss: 0.2854, Accuracy: 0.8986
Epoch 18/100, Loss: 0.2988, Accuracy: 0.8855
Epoch 19/100, Loss: 0.3193, Accuracy: 0.8906
Epoch 20/100, Loss: 0.3188, Accuracy: 0.8819
Epoch 21/10